In [2]:
import sys
import os 
import pandas as pd 
from Bio import SeqIO
from tqdm import tqdm
import json


path_project = "/media/concha-eloko/Linux/depolymerase_building"

phagedepo_df = pd.read_csv(f"{path_project}/Dpo_domains.phagedepo.0805.final.tsv", sep = "\t", header = 0)
phagedepo_df

,Seq_ID,Fold,Prob,Boundaries,Dpo_seq,Full_seq
0,phagedepo__3863,right-handed beta-helix,1.000,3863_A_6_164_549,VVSVLDFYLGTDPDYTNALNRALTASNGVLVPQGEYSTSLLSHPTC...,MSSGCGDVLSLADLQTAKKHQIFEAEVITGKSGGVAGGADIDYATN...
1,phagedepo__4467,right-handed beta-helix,1.000,4467_A_5_294_636,GVNVLDFMTFAERAAALNYTGTNDNSEAFRKALATGSRQIIAPPGR...,MDQEIKTVIQYPTGSTEFDIPFDYLSRKFVRVSLVSDDNRRLLSNI...
2,phagedepo__4576,right-handed beta-helix,1.000,4576_A_3_184_534,ELKSARFYGVSSTRTDNAVQLNAAANSGGRVVIPGGVTLFNSEILT...,MADILVTSPYRPFTLPNQFKAVFNGSIYCGTVDAVDPSNSQVQVYK...
3,phagedepo__5510,right-handed beta-helix,0.997,5510_A_5_278_575,QSISSTLEIRECTGVEVHRASGLMACFLFRGCHFCKMVDADNPSGG...,MSSGCGDVLSLNDLQIAKKHQIFEAEVITGKQGGVAGGADIDYATN...
4,phagedepo__6442,right-handed beta-helix,0.995,6442_A_6_361_681,PDNVKNQNLISHLIISNSSGIDVFYPKATFGSYESFKNNNVKFWYP...,MTVSIEVDHNDYIGNGVTTSFPYTFRIFKKSDLVVQVADLSENITE...
...,...,...,...,...,...,...
699,phagedepo__4646,right-handed beta-helix,1.000,4646_A_5_379_783,PTATDILSSNGKTLQYLTDKVNQVYDVKDFGAVGDGTTDDTAAVQS...,MSVGLYGDGVSESQENINVTQYGWTNEDVAGITLIQDYLNQIQTLF...
700,phagedepo__6447,right-handed beta-helix,1.000,6447_A_5_337_699,DRVNPFMFGALGGSNDDTIPVQSCVDSGKATQLTDAHYVSNIQLKY...,MGYFQMTRNVEELFGGVITAPHQIPFTYKSNVGGETFLSLPFYPVT...
701,phagedepo__1994,right-handed beta-helix,1.000,1994_A_5_283_679,PISWATQFGATKDGGREDVINAAIARMSQWSSFGIDPGIKFNNPLI...,MSDCKSYVSKEDLQALKESQQHIEHVARSRNAAGEKALQVTDAIRG...
702,phagedepo__4581,right-handed beta-helix,0.872,full_protein,MLQVKDFSGATHAEQIQNAINAASTSALHKTVQLEENKDYSITAPI...,MLQVKDFSGATHAEQIQNAINAASTSALHKTVQLEENKDYSITAPI...


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments

# Replace <model_checkpoint> with the path to the ESM-2 model checkpoint or the name of the model if you want to use a pre-trained one.
model_checkpoint = "<model_checkpoint>"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=3)


In [ ]:
import pandas as pd

# Replace <path_to_dataset> with the path to your dataset file.
dataset = pd.read_csv("<path_to_dataset>")
# Rename the column containing the labels to "labels".
dataset = dataset.rename(columns={"<column_name>": "labels"})

# Tokenize the input data.
tokenized_inputs = tokenizer(
    list(dataset["input_text"]),
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt"
)

# Convert the labels to a list of integers.
labels = list(dataset["labels"])

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=2e-5,
    evaluation_strategy="steps",
    eval_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_inputs,
    train_labels=labels,
)

In [ ]:
trainer.train()

In [ ]:
# Replace <path_to_validation_set> with the path to your validation set file.
validation_set = pd.read_csv("<path_to_validation_set>")
validation_set = validation_set.rename(columns={"<column_name>": "labels"})

# Tokenize the validation set inputs.
tokenized_validation_inputs = tokenizer(
    list(validation_set["input_text"]),
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt"
)

# Convert the validation set labels to a list of integers.
validation_labels = list(validation_set["labels"])

# Evaluate the model on the validation set.
eval_results = trainer.evaluate(tokenized_validation_inputs, validation_labels)
print(eval_results)

In [ ]:
# Replace <path_to_saved_model> with the path where you want to save the fine-tuned model.
trainer.save_model("<path_to_saved_model>")